# Part 7 Forward modelling and optimization of soil properties using the C- and X-Band radar backscatter data.
[Benoit Montpetit](https://github.com/ecccben), *CPS/CRD/ECCC*, 2024  
[Mike Brady](https://github.com/m9brady), *CPS/CRD/ECCC*, 2024

Here, we use the C- (Radarsat-2) and X-Band (TerraSAR-X) radar backscatter data to optimize the background soil properties using SMRT and the Geometrical Optics background surface scattering model.

In [ ]:
import geopandas as gp
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.optimize import least_squares
from scipy.interpolate import interp1d
from smrt.core.globalconstants import PERMITTIVITY_OF_AIR
from smrt import sensor_list, make_soil, make_model, make_snowpack
from matplotlib import pyplot as plt
import matplotlib

font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)
plt.rcParams["axes.labelsize"] = 22
plt.rcParams["axes.labelweight"] = 'bold'
plt.rcParams['xtick.labelsize']=16
plt.rcParams['ytick.labelsize']=16

pd.set_option('use_inf_as_na', True)

from radarfunc import to_dB

Extrapolation function of permittivity implemented in SMRT based on Montpetit et al. (2018)

In [ ]:
def extra_permi(f):
    
    p = interp1d([10.65e9, 19e9, 37e9],
                 [complex(3.18, 0.0061), complex(3.42, 0.0051), complex(4.47, 0.33)],
                 fill_value="extrapolate")
    
    return p(f)

# Function that uses the snow and soil inputs to simulate the backscatter at the snow surface using SMRT

In [ ]:
def smrtSim(f, theta, thickness, density, temperature, p_ex, grain_type, soil_params, bands=['vv']):
    
    '''
    param2opti: list of three parameters [epsr, epsi, phi]
        epsr: real part of the soil permi[epsr, epsi, phi]
        epsr: real part of the soil permittivity
        epsi: imaginary part of the soil permittivity
        phi: correction factor to apply to snow microstructure
    sig_soil: soil roughness root-mean-square height (in m)
    lc_soil: soil roughness correlation length (in m)
    t_soil: soil surface temperature (in K)
    snowpit: SMRT snowpit measurements
        layer:
            thickness: snow thickness per layer (in m)
            density: snow density per layer (in m)
            temperature: snow temperature per layer (in K)
        microstructure:
            corr_length: snow correlation length per layer (in m)
    pols: list of polarizations to model using SMRT ['VV', 'HH', 'HV', 'VH']
    sat_sig0: measured sigma-nought values from satellite/airborne sensors (in linear units)
    '''
    
    #Modeling theories to use in SMRT
    model = make_model("iba", "dort", rtsolver_options = {'m_max': 2, 'error_handling':'nan', 'phase_normalization' : True})
    
    #Initializing the modeled sigma-nought values
    sim_sig0 = {}
    
    #Creating the substrate under the snowpack
    '''
    Roughness model: Geometrical Optics Backscatter model
    Permittivity model: static complexe permittivity values to optimize
    Mean_square_slope: Mean Square Slope calculated previously (mss, no units)
    Temperature: Soil temperature (in K)
    '''

    sub = make_soil('geometrical_optics', 
                    permittivity_model = complex(soil_params[0],soil_params[1]), 
                    mean_square_slope=soil_params[2], 
                    temperature = 253.7)
    
    kappa = np.array([0.75] * len(grain_type))
    kappa[np.where(grain_type=='H')[0]] = 1.3
    
    #Creating the snowpack to simulate with the substrate
    sp = make_snowpack(thickness=thickness, 
                       microstructure_model='exponential',
                       density= density,
                       temperature=temperature,
                       ice_permittivity_model=None,
                       background_permittivity_model=PERMITTIVITY_OF_AIR,
                       liquid_water=0, salinity=0, 
                       corr_length = kappa * p_ex,
                       substrate = sub)

    #Initializing the outputs dataframe
    sim_sig0=pd.DataFrame({'theta': [], 'sig0': []})

    sensor  = sensor_list.active(f, theta)

    sigma_nought = model.run(sensor, sp, parallel_computation=True)
    
    for tet in theta:
    
        for band in bands:

            sim_sig0 = pd.concat([sim_sig0,pd.DataFrame({'theta': [np.array(tet)], 
                                                         'pol':[band],
                                                         'sig0': [sigma_nought.sigma_as_dataframe().loc[
                                                                  tet,band[0].upper(),band[1].upper()].sigma]})], 
                                 ignore_index=True)

    return sim_sig0

# Optimization function that calculates and returns the difference between the simulated backscatter from SMRT and the measured backscatter  
This delta will then be used by the least_squares function to retrieve the optimal parameters

In [ ]:
def opti_cx(params2opti, snowpits, sig0, thetas, fs, sites):
    
    '''
    param2opti: list of five parameters [epsr_c, epsi_c, epsr_x, epsi_x, mss]
        epsr_c: real part of the soil permittivity at C-Band
        epsi_c: imaginary part of the soil permittivity at C-Band
        epsr_x: real part of the soil permittivity at X-Band
        epsi_x: imaginary part of the soil permittivity at X-Band
        mss: Mean square slope roughness parameter used in the Geo Optics model
    snowpit: SMRT snowpit measurements
        layer:
            thickness: snow thickness per layer (in m)
            density: snow density per layer (in m)
            temperature: snow temperature per layer (in K)
        microstructure:
            corr_length: snow correlation length per layer (in m)
    sig0: measured sigma-nought values from satellite/airborne sensors (in linear units)
    thetas: measured local incidence angles
    fs: frequency of the radar signal
    sites: list of sites measured
    '''
    
    #Initializing the modeled sigma-nought values
    sim_sig0 = pd.DataFrame(sig0)
    
    #Initializing the difference between the measurements and the simulations to optimize
    delta = np.array([])

    for f in fs.unique():

        if f == 9.65e9:

            param2opti = params2opti[2:]

        elif f == 5.405e9:

            param2opti = np.concatenate((params2opti[:2],params2opti[-1]),axis=None)

    
        for site in sites.unique():
            
            theta = list(thetas.loc[(sites==site) & (fs==f)].drop_duplicates())

            sim_sig0_temp = smrtSim(f, 
                                    theta, 
                                    snowpits[site]['thickness'], 
                                    snowpits[site]['density'], 
                                    snowpits[site]['temperature'], 
                                    snowpits[site]['p_ex'],
                                    snowpits[site]['grain_type'], 
                                    param2opti)

            for k in range(len(theta)):

                # if not sim_sig0_temp.loc[(sim_sig0_temp['theta']==theta[k]), 'sig0'].isnull().any():
                    
                sim_sig0.loc[(thetas==theta[k]) & (fs==f) & (sites==site),
                        'loc_sig0'] = sim_sig0_temp.loc[(sim_sig0_temp['theta']==theta[k]), 'sig0'].values[0]

                
            delta=np.append(delta,(to_dB(sim_sig0['loc_sig0'].dropna().values) - to_dB(sig0.loc[(sim_sig0['loc_sig0'].isnull()==False)].values)))
                        
                    

    print('epsr_c = ' + "{:.2f}".format(params2opti[0]) + 
        '; epsi_c = ' + "{:.4f}".format(params2opti[1]) + 
        '; epsr_x = ' + "{:.2f}".format(params2opti[2]) + 
        '; epsi_x = ' + "{:.4f}".format(params2opti[3]) + 
        '; mss = ' + "{:.3f}".format(params2opti[4]) +
        '; lsq = ' + "{:.3f}".format(np.sum(delta**2)))

    return delta

Initial parameters

In [ ]:
from constants import TVC02
sites = pd.DataFrame({'site':TVC02})
sites.site.replace({'RS':'RP'}, regex=True, inplace=True)
sites=list(sites.site.values)
sites.remove('SC02')

In [ ]:
fs = [5.405e9,9.65e9]

eps = extra_permi(fs)

soil_rough = pd.read_pickle('../Data/SoilRoughness_ALS2018_TVC18-19_DB.pkl')
rmsh = soil_rough.loc[soil_rough.mss<0.04].rms.median() #Based on 2018 LiDAR point cloud
lc = soil_rough.loc[soil_rough.mss<0.04].lc.median() #Based on 2018 LiDAR point cloud

mss = 2*(rmsh/lc)**2

params2opti = [np.real(eps[0]), np.imag(eps[0]), np.real(eps[1]), np.imag(eps[1]), mss]
lbound = [2, 0.0001, 2, 0.0001, 0.001]
ubound = [5, 0.5, 5, 0.5, 0.05]        
bounds = (lbound, ubound)

bands = ['vv']

In [ ]:
CBand = gp.read_file('../Data/RS2_TVC18-19_DB.geojson').dropna()
XBand = gp.read_file('../Data/TSX_TVC18-19_DB.geojson').dropna()
CBand = CBand[(CBand['Date']>=datetime(2019,1,1)) & (CBand['Date']<datetime(2019,3,1))]
XBand = XBand[(XBand['Date']>=datetime(2019,1,1)) & (XBand['Date']<datetime(2019,3,1))]
CBand = CBand.query('Station.isin(@sites)')
CBand = CBand.query('pol.isin(@bands)')
XBand = XBand.query('Station.isin(@sites)')
XBand = XBand.query('pol.isin(@bands)')
sig0 = pd.concat([CBand['loc_sig0'], XBand['loc_sig0']], ignore_index=True)
pols = pd.concat([CBand['pol'], XBand['pol']], ignore_index=True)
thetas = pd.concat([CBand['loc_inc_ang'], XBand['loc_inc_ang']], ignore_index=True)
sites = pd.concat([CBand['Station'], XBand['Station']], ignore_index=True)
fs = pd.DataFrame({'f':np.concatenate((np.ones(len(CBand['loc_sig0']))*5.405e9, np.ones(len(XBand['loc_sig0']))*9.65e9),axis=None)})['f']

snowpits = pd.read_pickle('../Data/df_pits_mode.pkl')

Running the least squares optimization

In [ ]:
results_cx = least_squares(opti_cx, params2opti, 
                          args=(snowpits, sig0, thetas, fs, sites), 
                          bounds=bounds,
                          ftol=1e-2,
                          method='trf')

In [ ]:
df_cx = pd.DataFrame({'site':['all'], 'epsr_c':[results_cx.x[0]], 'epsi_c':[results_cx.x[1]], 
                      'epsr_x':[results_cx.x[2]], 'epsi_x':[results_cx.x[3]], 'mss': [results_cx.x[4]], 'lsq':[results_cx.cost]})
df_cx.to_pickle('../Data/TVC_background_CX-Band.pkl')

In [ ]:
for site in sites.unique():
    
    print(site)

    results = least_squares(opti_cx, params2opti, 
                            args=(snowpits, sig0[sites==site], thetas[sites==site], fs, sites[sites==site]), 
                            bounds=bounds,
                            ftol=1e-2,
                            method='trf')
    
    df_cx= pd.concat([df_cx, pd.DataFrame({'site':[site], 'epsr_c':[results.x[0]], 'epsi_c':[results.x[1]], 
                      'epsr_x':[results.x[2]], 'epsi_x':[results.x[3]], 'mss': [results.x[4]], 'lsq':[results.cost]})], ignore_index=True)

In [ ]:
df_cx.to_pickle('../Data/TVC_background_CX-Band.pkl')
df_cx

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site</th>
      <th>epsr_c</th>
      <th>epsi_c</th>
      <th>epsr_x</th>
      <th>epsi_x</th>
      <th>mss</th>
      <th>lsq</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>all</td>
      <td>2.466008</td>
      <td>0.004518</td>
      <td>2.606789</td>
      <td>0.006096</td>
      <td>0.010198</td>
      <td>5984.775885</td>
    </tr>
    <tr>
      <th>1</th>
      <td>RP16</td>
      <td>2.068339</td>
      <td>0.000499</td>
      <td>2.187264</td>
      <td>0.001095</td>
      <td>0.009505</td>
      <td>15.118812</td>
    </tr>
    <tr>
      <th>2</th>
      <td>RP17</td>
      <td>2.638995</td>
      <td>0.003530</td>
      <td>2.762957</td>
      <td>0.003322</td>
      <td>0.009669</td>
      <td>7.158915</td>
    </tr>
    <tr>
      <th>3</th>
      <td>RP18</td>
      <td>2.580436</td>
      <td>0.003229</td>
      <td>2.755940</td>
      <td>0.004118</td>
      <td>0.009661</td>
      <td>4.049906</td>
    </tr>
    <tr>
      <th>4</th>
      <td>RP19</td>
      <td>3.029246</td>
      <td>0.006728</td>
      <td>3.149412</td>
      <td>0.006220</td>
      <td>0.016802</td>
      <td>0.023757</td>
    </tr>
    <tr>
      <th>5</th>
      <td>RP20</td>
      <td>2.402191</td>
      <td>0.002483</td>
      <td>2.541286</td>
      <td>0.002977</td>
      <td>0.008981</td>
      <td>5.793995</td>
    </tr>
    <tr>
      <th>6</th>
      <td>RP21</td>
      <td>2.562950</td>
      <td>0.003417</td>
      <td>2.659355</td>
      <td>0.003605</td>
      <td>0.010581</td>
      <td>4.276721</td>
    </tr>
    <tr>
      <th>7</th>
      <td>RP22</td>
      <td>2.552231</td>
      <td>0.003429</td>
      <td>2.589926</td>
      <td>0.003236</td>
      <td>0.011585</td>
      <td>3.376985</td>
    </tr>
    <tr>
      <th>8</th>
      <td>RP23</td>
      <td>2.520022</td>
      <td>0.003225</td>
      <td>2.528385</td>
      <td>0.002909</td>
      <td>0.008735</td>
      <td>4.078824</td>
    </tr>
    <tr>
      <th>9</th>
      <td>RP24</td>
      <td>2.396187</td>
      <td>0.002263</td>
      <td>2.550421</td>
      <td>0.003154</td>
      <td>0.007307</td>
      <td>7.587102</td>
    </tr>
    <tr>
      <th>10</th>
      <td>RP25</td>
      <td>2.280866</td>
      <td>0.001550</td>
      <td>2.492229</td>
      <td>0.002649</td>
      <td>0.009302</td>
      <td>5.336857</td>
    </tr>
    <tr>
      <th>11</th>
      <td>RP26</td>
      <td>2.463236</td>
      <td>0.001976</td>
      <td>2.600816</td>
      <td>0.003294</td>
      <td>0.009951</td>
      <td>4.996285</td>
    </tr>
    <tr>
      <th>12</th>
      <td>RP27</td>
      <td>2.258602</td>
      <td>0.001594</td>
      <td>2.258805</td>
      <td>0.001476</td>
      <td>0.007965</td>
      <td>2.357273</td>
    </tr>
    <tr>
      <th>13</th>
      <td>RP28</td>
      <td>2.741816</td>
      <td>0.004625</td>
      <td>2.778596</td>
      <td>0.004239</td>
      <td>0.012673</td>
      <td>8.556674</td>
    </tr>
    <tr>
      <th>14</th>
      <td>RP29</td>
      <td>2.603640</td>
      <td>0.003607</td>
      <td>2.740354</td>
      <td>0.004036</td>
      <td>0.013251</td>
      <td>5.667158</td>
    </tr>
    <tr>
      <th>15</th>
      <td>RP30</td>
      <td>2.652969</td>
      <td>0.004729</td>
      <td>2.885955</td>
      <td>0.005293</td>
      <td>0.013267</td>
      <td>4.351416</td>
    </tr>
    <tr>
      <th>16</th>
      <td>RP31</td>
      <td>2.295109</td>
      <td>0.001829</td>
      <td>2.596983</td>
      <td>0.003273</td>
      <td>0.008949</td>
      <td>2.740762</td>
    </tr>
    <tr>
      <th>17</th>
      <td>SD02</td>
      <td>2.628416</td>
      <td>0.002634</td>
      <td>2.948112</td>
      <td>0.005140</td>
      <td>0.010893</td>
      <td>11.317456</td>
    </tr>
    <tr>
      <th>18</th>
      <td>SM02</td>
      <td>2.322261</td>
      <td>0.001816</td>
      <td>2.443560</td>
      <td>0.002458</td>
      <td>0.010899</td>
      <td>7.389861</td>
    </tr>
    <tr>
      <th>19</th>
      <td>SO02</td>
      <td>2.543608</td>
      <td>0.003346</td>
      <td>2.395109</td>
      <td>0.002200</td>
      <td>0.010224</td>
      <td>2.370258</td>
    </tr>
    <tr>
      <th>20</th>
      <td>ST02</td>
      <td>2.270314</td>
      <td>0.001693</td>
      <td>2.381293</td>
      <td>0.002127</td>
      <td>0.008869</td>
      <td>3.952208</td>
    </tr>
    <tr>
      <th>21</th>
      <td>SV02</td>
      <td>2.499894</td>
      <td>0.003228</td>
      <td>2.503828</td>
      <td>0.002810</td>
      <td>0.010368</td>
      <td>2.797720</td>
    </tr>
  </tbody>
</table>
</div>

Computing the standard deviations for the sites individually

In [ ]:
df_cx.iloc[1:,1:].std()

epsr_c    0.207719  
epsi_c    0.001369  
epsr_x    0.228339  
epsi_x    0.001235  
mss       0.002150  
lsq       3.353872  
dtype: float64

In [ ]:
#Initializing the modeled sigma-nought values
sim_sig0_cx = pd.DataFrame(sig0)

params = df_cx.loc[df_cx.site=='all']

for f in fs.unique():
    
    for site in sites.unique():

        if f == 9.65e9:

            param = params.iloc[0,3:-1].values

        elif f == 5.405e9:

            param = np.concatenate((params.iloc[0,1:3].values,params.iloc[0,-2]),axis=None)

        theta = list(thetas.loc[(sites==site) & (fs==f)].drop_duplicates())

        sim_sig0_temp = smrtSim(f, 
                                theta, 
                                snowpits[site]['thickness'], 
                                snowpits[site]['density'], 
                                snowpits[site]['temperature'], 
                                snowpits[site]['p_ex'],
                                snowpits[site]['grain_type'], 
                                list(param))

        for k in range(len(theta)):

            sim_sig0_cx.loc[(thetas==theta[k]) & (fs==f) & (sites==site),
                        'loc_sig0'] = sim_sig0_temp.loc[(sim_sig0_temp['theta']==theta[k]), 'sig0'].values[0]

In [ ]:
#Initializing the modeled sigma-nought values
sim_sig0_cx_sites = pd.DataFrame(sig0)

for f in fs.unique():
    
    for site in sites.unique():

        params = df_cx.loc[df_cx.site==site]

        if f == 9.65e9:

            param = params.iloc[0,3:-1].values

        elif f == 5.405e9:

            param = np.concatenate((params.iloc[0,1:3].values,params.iloc[0,-2]),axis=None)

        theta = list(thetas.loc[(sites==site) & (fs==f)].drop_duplicates())

        sim_sig0_temp = smrtSim(f, 
                                theta, 
                                snowpits[site]['thickness'], 
                                snowpits[site]['density'], 
                                snowpits[site]['temperature'], 
                                snowpits[site]['p_ex'],
                                snowpits[site]['grain_type'], 
                                list(param))

        for k in range(len(theta)):

            sim_sig0_cx_sites.loc[(thetas==theta[k]) & (fs==f) & (sites==site),
                        'loc_sig0'] = sim_sig0_temp.loc[(sim_sig0_temp['theta']==theta[k]), 'sig0'].values[0]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

epsr_c_hist = ax.hist(df_cx.iloc[1:].epsr_c, bins=np.arange(2,4.2,0.2), density=True, label = 'C-Band', color='k')
epsr_x_hist = ax.hist(df_cx.iloc[1:].epsr_x, bins=np.arange(2,4.2,0.2), density=True, label = 'X-Band', alpha=0.5,color='grey')

ax.legend()
ax.set_xlabel('$\\epsilon\'_{soil}$', fontsize=20, fontweight='bold')
ax.set_ylabel('Probability Density Function', fontsize=20, fontweight='bold')

<center><img src="Figures/Figure10.png" Width="1000px"></center>

Figure 10 of [Montpetit et al., (2024)](link TBD): Distributions of the retrieved real $\varepsilon_{soil}$ for C- and X-Band data for all the sites measured during the January campaign.

In [ ]:
pd.DataFrame({'RMSE_sites':[np.sqrt(np.nanmean((to_dB(sim_sig0_cx_sites.loc[fs==5.405e9,'loc_sig0'].values)-
                                      to_dB(sig0.loc[(fs==5.405e9)].values))**2)),
                      np.sqrt(np.nanmean((to_dB(sim_sig0_cx_sites.loc[fs==9.65e9,'loc_sig0'].values)-
                                to_dB(sig0.loc[(fs==9.65e9)].values))**2)),
                      np.sqrt(np.nanmean((to_dB(sim_sig0_cx_sites.loc[:,'loc_sig0'].values)-
                                to_dB(sig0.values))**2))],
              'Bias_sites':[np.nanmean((to_dB(sim_sig0_cx_sites.loc[fs==5.405e9,'loc_sig0'].values)-
                              to_dB(sig0.loc[(fs==5.405e9)].values))),
                      np.nanmean((to_dB(sim_sig0_cx_sites.loc[fs==9.65e9,'loc_sig0'].values)-
                        to_dB(sig0.loc[(fs==9.65e9)].values))),
                      np.nanmean((to_dB(sim_sig0_cx_sites.loc[:,'loc_sig0'].values)-
                        to_dB(sig0.values)))],
               'RMSE_all':[np.sqrt(np.nanmean((to_dB(sim_sig0_cx.loc[fs==5.405e9,'loc_sig0'].values)-
                                     to_dB(sig0.loc[fs==5.405e9].values))**2)),
                           np.sqrt(np.nanmean((to_dB(sim_sig0_cx.loc[fs==9.65e9,'loc_sig0'].values)-
                                     to_dB(sig0.loc[fs==9.65e9].values))**2)),
                           np.sqrt(np.nanmean((to_dB(sim_sig0_cx.loc[:,'loc_sig0'].values)-
                                     to_dB(sig0.values))**2))],
               'Bias_all':[np.nanmean((to_dB(sim_sig0_cx.loc[fs==5.405e9,'loc_sig0'].values)-
                             to_dB(sig0.loc[fs==5.405e9].values))),
                           np.nanmean((to_dB(sim_sig0_cx.loc[fs==9.65e9,'loc_sig0'].values)-
                             to_dB(sig0.loc[fs==9.65e9].values))),
                           np.nanmean((to_dB(sim_sig0_cx.loc[:,'loc_sig0'].values)-
                             to_dB(sig0.values)))]}, 
                           index=['C-Band','X-Band','All'])

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>RMSE_sites</th>
      <th>Bias_sites</th>
      <th>RMSE_all</th>
      <th>Bias_all</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>C-Band</th>
      <td>0.685254</td>
      <td>0.017930</td>
      <td>1.180711</td>
      <td>0.055724</td>
    </tr>
    <tr>
      <th>X-Band</th>
      <td>0.280577</td>
      <td>-0.000347</td>
      <td>0.809470</td>
      <td>0.119933</td>
    </tr>
    <tr>
      <th>All</th>
      <td>0.622270</td>
      <td>0.014079</td>
      <td>1.112836</td>
      <td>0.069253</td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

ax[0].plot([-20,-6],[-20,-6],'-k',alpha=0.5)
ax[0].plot(to_dB(sim_sig0_cx.loc[fs==5.405e9].dropna().values),to_dB(sig0.loc[(fs==5.405e9) & (sim_sig0_cx_sites['loc_sig0'].isnull()==False)].values),'.k',markersize=20, label='C-Band')
ax[0].plot(to_dB(sim_sig0_cx.loc[fs==9.65e9].values),to_dB(sig0.loc[fs==9.65e9].values),'.',color='grey', alpha=0.75,markersize=20, label='X-Band')
ax[0].text(-19.75,-6.5,'a)')

ax[1].plot([-20,-6],[-20,-6],'-k',alpha=0.5)
ax[1].plot(to_dB(sim_sig0_cx_sites.loc[fs==5.405e9].values),to_dB(sig0.loc[fs==5.405e9].values),'.k',markersize=20, label='C-Band')
ax[1].plot(to_dB(sim_sig0_cx_sites.loc[fs==9.65e9].values),to_dB(sig0.loc[fs==9.65e9].values),'.',color='grey', alpha=0.75,markersize=20,label='X-Band')
ax[1].text(-19.75,-6.5,'b)')

for i in range(2):

    ax[i].set_xlim(-20,-6)
    ax[i].set_ylim(-20,-6)
    ax[i].set_xlabel('$\sigma^0_{mes}$',fontsize=20, fontweight='bold')

    if i==0:
        ax[i].set_ylabel('$\sigma^0_{sim}$',fontsize=20, fontweight='bold')

ax[1].legend(loc=4)

<center><img src="../Figures/Figure11.png" Width="1000px"></center>

Figure 11 of [Montpetit et al., (2024)](link TBD): Comparison between simulated and measured $\sigma^0$ at C- and X-Band for a) using a single set of parameters for all sites (’All’ in
Table 6) and b) retrieved parameters for each site individually (distributed values shown in Figure 10).